# Tienda de audiolibros

## Problema

Este problema trata sobre predecir si un cliente volverá a realizar una compra en una tienda de audiolibros en base a su comportamiento histórico con la tienda.

Para lo anterior se cuenta con una base de datos que cuenta con 2 años de recolección de datos de cada cliente, y para ver si un cliente volvió a comprar en la tienda se utilizaron 6 meses adicionales de observación.

La base de datos cuenta con la siguiente informacion de cada cliente:
<ul>
    <li>ID de cliente</li>
<li>Suma de duración en minutos de todos sus libros</li>

<li>Duracion promedio de cada libro</li>

<li>Suma de dinero total pagada por todos los libros</li>

<li>Precio promedio pagado por cada libro</li>

<li>Review(1 si el cliente dejó review, 0 en caso contrario)</li>

<li>Nota de review promedio de libros (escala de 1 al 10)</li>

<li>Minutos totales escuchados</li>

<li>Porcentaje completado del último libro</li>

<li>Tiempo transcurrido desde la primera compra y última vez que abrió el libro</li>
    
<li>Target (1 si compró en los ultimos 6 meses, 0 en caso contrario) ------------ Variable a predecir</li>
 
</ul>

En este proyecto se construye un algoritmo de Machine Learning utilizando Redes Neuronales capáz de predecir si un cliente volverá a comprar durante los próximos 6 meses.

## Preprocesamiento de los datos

### Importando las librerías relevantes

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
# Utilizaremos datos sin encabezados para trabajar con nuestro problema.
datos_primarios = np.loadtxt('Audiobooks_data.csv',delimiter=',')

# Utilizaremos datos con encabezados para hacerse una idea de la tabla original
datos_primarios_con_encabezados = pd.read_csv('Audiobooks_data_con_encabezados.csv', sep = ';')
datos_primarios_con_encabezados.head()

,ID,Book length (mins)_overall,Book length (mins)_avg,Price_overall,Price_avg,Review,Review 10/10,Minutes listened,Completion,Support Requests,Last visited minus Purchase date,Targets
0,873,2160,2160,10.13,10.13,0,8.91,0.0,0.0,0,0,1
1,611,1404,2808,6.66,13.33,1,6.50,0.0,0.0,0,182,1
2,705,324,324,10.13,10.13,1,9.00,0.0,0.0,1,334,1
3,391,1620,1620,15.31,15.31,0,9.00,0.0,0.0,0,183,1
4,819,432,1296,7.11,21.33,1,9.00,0.0,0.0,0,0,1


In [3]:
# De los datos primarios podemos obtener nuestros datos de input al eliminar las columnas de 'ID' y 'Targets'
# Asi generamos nuestra tabla de inputs
inputs_sin_escalar_todos = datos_primarios[:,1:-1]

In [4]:
# De los datos primarios podemos obtener los targets que se encuentran en la última columna
targets_todos = datos_primarios[:,-1]

### Balanceando el dataset

Balancear el dataset significa contar con una cantidad equitativa de muestras cuyos Targets sean 50% valor 1 y 50% valor 0.
De esta forma el modelo puede aprender a diferenciar de mejor forma cada caso.

In [5]:
# Contamos cuantos targets del dataset tienen valor 1.
targets_valor_uno = int(np.sum(targets_todos))

# Establecemos un contador para Targets con valor 0
targets_valor_cero = 0 

# Como queremos crear un dataset balanceado, tenemos que remover algunas filas
indices_a_remover = []

# Recorreremos el dataset con un ciclo for
# Contamos el numero de Targets igual a 0
# Una vez tengamos tantos 0s como 1s, marcamos los indices donde los targets son 0
for i in range(targets_todos.shape[0]):
    if targets_todos[i] == 0:
        targets_valor_cero += 1
        if targets_valor_cero > targets_valor_uno:
            indices_a_remover.append(i)

# Creamos 2 nuevas variables, una que contendrá los inputs y otra que contendrá los targets
# Borramos todos los índices que marcamos para remover en el ciclo for
inputs_sin_escalar_balanceados = np.delete(inputs_sin_escalar_todos, indices_a_remover, axis=0)
targets_balanceados = np.delete(targets_todos, indices_a_remover, axis=0)

### Estandarizamos los inputs

In [6]:
inputs_escalados = preprocessing.scale(inputs_sin_escalar_balanceados)

### Mezclando los datos

In [7]:
# Mezclaremos los datos, puestos que estos fueron recogidos en orden de fecha
indices_mezclados = np.arange(inputs_escalados.shape[0])
np.random.shuffle(indices_mezclados)

# Usamos los índices mezclados para mezclar los inputs y targets de manera independiente
inputs_mezclados = inputs_escalados[indices_mezclados]
targets_mezclados = targets_balanceados[indices_mezclados]

### Dividiento el dataset en entrenamiento, validación y testeo

In [8]:
# Contamos el numero total de muestras
numero_de_muestras = inputs_mezclados.shape[0]

# Queremos repartir los datos de la forma 80% datos entrenamiento, 10% datos de validación y 10% datos de testeo

numero_muestras_entrenamiento = int(0.8 * numero_de_muestras)
numero_muestras_validacion = int(0.1 * numero_de_muestras)

# El dataset de testeo contiene todos los datos restantes
numero_muestras_test = numero_de_muestras - numero_muestras_entrenamiento - numero_muestras_validacion

# Creamos inputs y targets de entrenamiento
inputs_entrenamiento = inputs_mezclados[:numero_muestras_entrenamiento]
targets_entrenamiento = targets_mezclados[:numero_muestras_entrenamiento]

# Creamos inputs y targets de validacion
inputs_validacion = inputs_mezclados[numero_muestras_entrenamiento:numero_muestras_entrenamiento+numero_muestras_validacion]
targets_validacion = targets_mezclados[numero_muestras_entrenamiento:numero_muestras_entrenamiento+numero_muestras_validacion]

# Creamos inputs y targets de testeo
test_inputs = inputs_mezclados[numero_muestras_entrenamiento+numero_muestras_validacion:]
test_targets = targets_mezclados[numero_muestras_entrenamiento+numero_muestras_validacion:]

# Comprobamos si los datos de entrenamiento, validacion, y testeo estan balanceados

a= pd.DataFrame(np.array([[np.sum(targets_entrenamiento),
                           numero_muestras_entrenamiento,
                           np.sum(targets_entrenamiento) / numero_muestras_entrenamiento],
                          [np.sum(targets_validacion),
                           numero_muestras_validacion,
                           np.sum(targets_validacion) / numero_muestras_validacion],
                          [np.sum(test_targets),
                           numero_muestras_test,
                           np.sum(test_targets) / numero_muestras_test]]),
                   columns=['Suma targets valor 1', 'Número muestras', 'Proporcion de targets valor 1'])
a.rename(index={0: "Entrenamiento", 1: "Validación", 2: "Test"})

,Suma targets valor 1,Número muestras,Proporcion de targets valor 1
Entrenamiento,1807.0,3579.0,0.504890
Validación,216.0,447.0,0.483221
Test,214.0,448.0,0.477679


De la tabla se puede observar que la proporción de 1s y 0s es 50% y 50%

### Guardamos los datos en *.npz para trabajar con tensorflow

In [9]:
# Guardamos los datos en un formato npz para trabajarlos con Tensorflow

np.savez('Audiobooks_datos_entrenamiento', inputs=inputs_entrenamiento, targets=targets_entrenamiento)
np.savez('Audiobooks_datos_validacion', inputs=inputs_validacion, targets=targets_validacion)
np.savez('Audiobooks_datos_test', inputs=test_inputs, targets=test_targets)